In [ ]:
# scores is a function which takes names of two strategies and return related scores between those two strategies 
def scores(m1,m2):
    import numpy as np
    from itertools import combinations 
    from copy import deepcopy
    from sklearn.preprocessing import normalize



    def Reward(S1,S2):
        ep = 0
        S2 = deepcopy(S2[0] + S2[2] + S2[1] + S2[3])
        P0 = np.zeros((4,4))
        for i in range(8):
            for j in range(4):
                if i == 0:
                    P0[i,j] = abs(int(S1[j])-ep) * abs(int(S2[j])-ep)
                elif i == 1:
                    P0[i,j] = abs(int(S1[j])-ep) * (1-abs(int(S2[j])-ep))
                elif i == 2:
                    P0[i,j] = (1-abs(int(S1[j])-ep)) * abs(int(S2[j]))
                elif i == 3:
                    P0[i,j] = (1-abs(int(S1[j])-ep)) * (1-abs(int(S2[j])-ep))

        P = np.concatenate((P0,P0),axis = 1)
        P = np.concatenate((P,P),axis = 0)


        cc = np.array([1,1,1,1,0,0,0,0])
        cc = cc.reshape(1,8)
        ot = np.array([0,0,0,0,1,1,1,1])
        ot = ot.reshape(1,8)

        Q = np.transpose(np.concatenate((cc,ot,ot,ot,cc,ot,ot,ot)))

        P_f = P * Q


        return P_f


    # In[22]:


    def Strategy_Matrix(strategy_list = ['0000','0001','0010','0011','0100','0101',
                     '0110','0111','1000','1001','1010','1100',
                     '1011','1101','1110','1111'], b1 = 2, b2 = 1.2):

        l = len(strategy_list)

        pair_list_up = list(combinations(strategy_list, 2))

        pair_list_low = []
        for p in pair_list_up:
            pair_list_low.append((p[1],p[0]))


        bonus = np.array([b1-1,-1,b1,0,b2-1,-1,b2,0]).reshape(8,1)



        U = []
        for up in pair_list_up:
            p = Reward(up[0],up[1])

            w, v = np.linalg.eig(p)

            v = abs((v[:,np.where(w == 1)[0]])/(np.linalg.norm(v[:,np.where(w == 1)[0]],axis = 0,ord=1)))
            re = bonus * v
            r = np.mean(re[re!=0])
            U.append(r)

    ###############################

        L = []
        for low in pair_list_low:
            p = Reward(low[0],low[1])

            w, v = np.linalg.eig(p)

            v = abs((v[:,np.where(w == 1)[0]])/(np.linalg.norm(v[:,np.where(w == 1)[0]],axis = 0,ord=1)))
            re = bonus * v
            r = np.mean(re[re!=0])
            L.append(r)  

    ###############################       

        D = []
        for d in strategy_list:
            p = Reward(d,d)

            w, v = np.linalg.eig(p)

            v = abs((v[:,np.where(w == 1)[0]])/(np.linalg.norm(v[:,np.where(w == 1)[0]],axis = 0,ord=1)))
            re = bonus * v
            r = np.mean(re[re!=0])
            D.append(r)

    ###############################

        S = np.zeros((l,l))

        S[np.triu_indices(l,1)] = L
        S = np.transpose(S)
        S[np.triu_indices(l,1)] = U
        np.fill_diagonal(S,D)



        return S



    # In[21]:


    ### strategy = cc,cd,dc,dd
    strategy_list = ['0000','0001','0010','0011','0100','0101',
                     '0110','0111','1000','1001','1010','1100',
                     '1011','1101','1110','1111']
    score_matrix=np.nan_to_num(Strategy_Matrix())#the full matrix of scores between all strategies
    return score_matrix[m1,m1],score_matrix[m2,m2],score_matrix[m1,m2],score_matrix[m2,m1]

In [ ]:
#takes the position of a block and returns one of its neighboures depends on k
def neighbour(i,j,k): #K:right neighbour or left ones
    if k==0:
        hi=(i-1)%N
        hj=(j)%N
    if k==1:
        hi=i
        hj=(j-1)%N
    if k==2:
        hi=(i-1)%N
        hj=(j-1)%N
    if k==3:
        hi=(i-1)%N
        hj=(j+1)%N
    return hi,hj

In [ ]:
# compute the chance of birth based on the score
def probscore(s):
    ro=np.power(np.e,(betha*s))
    return ro

In [ ]:
# locates the prisoner strategie in the prison corner
def prison_line(prisoner):
    global l
    l=np.ones((N,N))
    repeat=0
    for i in range(N):
        for j in range(N):
            if repeat< prisoner:
                l[i,j]=-1
                repeat+=1
    

In [ ]:
def prison_square(prisoner):
    global l
    l=np.ones((N,N))
    repeat=0
    v=int(np.sqrt(prisoner))
    for i in range(v+1):
        for j in range(v):
            if repeat<prisoner:
                l[i,j]=-1
                repeat+=1

In [ ]:
def without_prison(prisoner):
    global l
    l=np.ones((N,N))
    for z in range(prisoner):
        l[np.random.randint(0,N)][np.random.randint(0,N)]=-1

In [ ]:
#finds the total score for each strategy
def sum_s(S):
    ind1,ind2=np.where(l==-1)
    sum_s2=0
    for i1,i2 in zip(ind1,ind2):
        sum_s2+=S[i1][i2]
    sum_total=np.sum(S)
    sum_s1=sum_total-sum_s2
    return sum_s1,sum_s2

In [ ]:
#gets the strategies and play a round with them, then compares the blocks and score each block in the score matrix
S=np.zeros((N,N))
def main(s1,s2,b):
    global S
    S=np.zeros((N,N))#the score matrix of all the blocks
    s11,s22,s12,s21=scores(s1,s2)
    for i in range(N):
        for j in range(N):
            for k in range(4):
                x,y=neighbour(i,j,k)
                if l[i,j]==1 and l[x,y]==1:
                    S[i,j]+=s11
                    S[x,y]+=s11
                    
                if l[i,j]==-1 and l[x,y]==-1:
                    S[i,j]+=s22
                    S[x,y]+=s22

                if l[i,j]==-1 and l[x,y]==1:
                    S[i,j]+=s21
                    S[x,y]+=s12

                if l[i,j]==1 and l[x,y]==-1:
                    S[i,j]+=s12
                    S[x,y]+=s21

    

    ro=np.power(np.e,np.array(b)*S)
    ro1,ro2=sum_s(ro)
    n1=(ro1/(ro1+ro2))*N**2
    
    if (n1-int(n1))>=0.5:
        r=np.random.random()
        if r>0.5:
            n1=int(n1)+1
        else:
            n1=int(n1)
    else:
        n1=int(n1)
    
    prison_line(N**2-n1)
    #prison_square(N**2-n1)
    #without_prison(N**2-n1)

    return n1
        
        

In [ ]:
########initial conditions

import numpy as np
import matplotlib.pyplot as plt
runtimes=100#number of games
betha= np.linspace(0.1 , 1.5 , 50)#learning
N=10 # size of lattice

##### n2 is number of invaders 
n2=1
###### n1 is the number of population
n1=N**2-n2

l=np.ones((N,N)) #the lattice of environment

In [ ]:

list_pop=[15]
list_ind=[0]
for b in betha:
    for m in list_pop:
        for n in list_ind:
            n1pop=[]
            l=np.ones((N,N))
            o=np.random.randint(0,N)
            u=np.random.randint(0,N)
            l[o][u]=-1
            s1=m#population
            s2=n#INVADOR
            for w in range(runtimes):
    #             print(w)
                n1=main(s1,s2,b)
            n1pop.append(n1)

n1pop=np.array(n1pop)
time=np.arange(0,len(n1pop))
plt.plot(betha,n1pop)
plt.show()

In [ ]:
print(S)